# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

## Load the Data

Start by importing the data from the pickle file.

In [2]:
# TODO: Implement load the data here.
with open('train.p', 'rb') as f:
    data = pickle.load(f)

In [3]:
print(data)

{'labels': array([ 0,  0,  0, ..., 42, 42, 42], dtype=uint8), 'features': array([[[[ 75,  78,  80],
         [ 74,  76,  78],
         [ 83,  84,  83],
         ..., 
         [ 67,  74,  73],
         [ 65,  69,  68],
         [ 66,  67,  66]],

        [[ 82,  83,  85],
         [ 80,  80,  82],
         [ 87,  86,  83],
         ..., 
         [ 73,  77,  77],
         [ 75,  77,  75],
         [ 79,  79,  77]],

        [[ 79,  79,  81],
         [ 84,  83,  85],
         [ 91,  88,  89],
         ..., 
         [ 72,  74,  72],
         [ 74,  75,  71],
         [ 78,  78,  74]],

        ..., 
        [[126, 128, 133],
         [118, 123, 128],
         [108, 117, 120],
         ..., 
         [ 96,  96,  90],
         [ 99, 101,  94],
         [ 99, 103,  99]],

        [[ 90,  94,  98],
         [ 91,  97, 103],
         [ 77,  87,  92],
         ..., 
         [104, 104,  97],
         [107, 111, 102],
         [ 95, 103,  97]],

        [[ 85,  84,  87],
         [ 93,  99, 1

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [4]:
# TODO: Use `train_test_split` here.
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], train_size=0.8)

In [5]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [6]:
# TODO: Implement data normalization here.

# Cast to a float type - currently integers
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Pixel range is 0-255, so divide by 255 to get to 0.0-1.0, then subtract 0.5 to get to the range -0.5 to +0.5
X_train = X_train / 255 - 0.5
X_val = X_val / 255 - 0.5

In [7]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [8]:
# TODO: Build a two-layer feedforward neural network with Keras here.

from keras.models import Sequential
from keras.layers.core import Dense, Activation

model = Sequential()
model.add(Dense(128, input_shape=(32*32*3,)))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 128)           393344      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 128)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 43)            5547        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 43)            0           dense_2[0][0]                    
Total params: 398,891
Trainable params: 398,891
Non-trainable params: 0
___________________

In [9]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [10]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_1 (None, 3072) (None, 128) <function linear at 0x7fe1c74be950>
activation_1 (None, 128) (None, 128) <function relu at 0x7fe1c74be730>
dense_2 (None, 128) (None, 43) <function linear at 0x7fe1c74be950>
activation_2 (None, 43) (None, 43) <function softmax at 0x7fe1c74be510>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [11]:
# TODO: Compile and train the model here.

from keras.optimizers import Adam  # import the Adam optimiser
from keras.utils import np_utils   # import some utils for the one-hot encoding

# Parameters
nb_classes = 43                    # number of classes
nb_epoch = 20                      # number of epochs in training (sneaked from the solution ... 2 above is a typo?)
batch_size = 512                   # batch size (not given, so picked a decent one...)

# One-hot encoding of labels
Y_train = np_utils.to_categorical(y_train, nb_classes)   # note the upper case Y_ notation
Y_val = np_utils.to_categorical(y_val, nb_classes)

# Reshape the inputs - need to flatten the images?
X_train_flat = X_train.reshape(-1, 32*32*3)
X_val_flat = X_val.reshape(-1, 32*32*3)

model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train_flat, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val_flat, Y_val))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 128)           393344      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 128)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 43)            5547        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 43)            0           dense_2[0][0]                    
Total params: 398,891
Trainable params: 398,891
Non-trainable params: 0
___________________

In [12]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9142

Training accuracy appears to be 0.9507

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [16]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

# Part 1: recreate the data - copy of code above to reset from the loaded 'data' variable
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], train_size=0.8)
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255 - 0.5
X_val = X_val / 255 - 0.5
# Now have the data back as far as the normalised state

# Create the model
from keras.models import Sequential
from keras.layers import Convolution2D, Dense, Activation, Flatten
from keras.optimizers import Adam  # import the Adam optimiser
from keras.utils import np_utils   # import some utils for the one-hot encoding

img_rows = 32
img_cols = 32
img_depth = 3
input_shape = (img_rows, img_cols, img_depth)       # parameter for the convolutional layer
kernel_size = (3, 3)                                # convolution kernel size
nb_filters = 32                                     # number of convolutional filters to use

model2 = Sequential()
model2.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model2.add(Activation('relu'))
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Activation('relu'))
model2.add(Dense(43))
model2.add(Activation('softmax'))

model2.summary()

# TODO: Compile and train the model here.

# Parameters
nb_classes = 43                    # number of classes
nb_epoch = 20                      # number of epochs in training (sneaked from the solution ... 2 above is a typo?)
batch_size = 512                   # batch size (not given, so picked a decent one...)

# One-hot encoding of labels
Y_train = np_utils.to_categorical(y_train, nb_classes)   # note the upper case Y_ notation
Y_val = np_utils.to_categorical(y_val, nb_classes)

# Compile the model
model2.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model2.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 30, 30, 32)    0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 28800)         0           activation_12[0][0]              
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 128)           3686528     flatten_4[0][0]                  
___________________________________________________________________________________________

In [17]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9762

Training accuracy was 0.9970

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [18]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

# Part 1: recreate the data - copy of code above to reset from the loaded 'data' variable
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], train_size=0.8)
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255 - 0.5
X_val = X_val / 255 - 0.5
# Now have the data back as far as the normalised state

# Create the model
from keras.models import Sequential
from keras.layers import Convolution2D, Dense, Activation, Flatten, MaxPooling2D
from keras.optimizers import Adam  # import the Adam optimiser
from keras.utils import np_utils   # import some utils for the one-hot encoding

img_rows = 32
img_cols = 32
img_depth = 3
input_shape = (img_rows, img_cols, img_depth)       # parameter for the convolutional layer
kernel_size = (3, 3)                                # convolution kernel size
nb_filters = 32                                     # number of convolutional filters to use
pool_size = (2, 2)                                  # size of pooling area for max pooling

model3 = Sequential()
model3.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=pool_size))
model3.add(Flatten())
model3.add(Dense(128))
model3.add(Activation('relu'))
model3.add(Dense(43))
model3.add(Activation('softmax'))

model3.summary()

# TODO: Compile and train the model here.

# Parameters
nb_classes = 43                    # number of classes
nb_epoch = 20                      # number of epochs in training (sneaked from the solution ... 2 above is a typo?)
batch_size = 512                   # batch size (not given, so picked a decent one...)

# One-hot encoding of labels
Y_train = np_utils.to_categorical(y_train, nb_classes)   # note the upper case Y_ notation
Y_val = np_utils.to_categorical(y_val, nb_classes)

# Compile the model
model3.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model3.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_5 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
activation_15 (Activation)       (None, 30, 30, 32)    0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 15, 15, 32)    0           activation_15[0][0]              
____________________________________________________________________________________________________
flatten_5 (Flatten)              (None, 7200)          0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [19]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9740

Training accuracy 0.9923

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [20]:
# TODO: Re-construct the network and add dropout after the pooling layer.

# Part 1: recreate the data - copy of code above to reset from the loaded 'data' variable
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], train_size=0.8)
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255 - 0.5
X_val = X_val / 255 - 0.5
# Now have the data back as far as the normalised state

# Create the model
from keras.models import Sequential
from keras.layers import Convolution2D, Dense, Activation, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam  # import the Adam optimiser
from keras.utils import np_utils   # import some utils for the one-hot encoding

img_rows = 32
img_cols = 32
img_depth = 3
input_shape = (img_rows, img_cols, img_depth)       # parameter for the convolutional layer
kernel_size = (3, 3)                                # convolution kernel size
nb_filters = 32                                     # number of convolutional filters to use
pool_size = (2, 2)                                  # size of pooling area for max pooling

model4 = Sequential()
model4.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model4.add(Activation('relu'))
model4.add(MaxPooling2D(pool_size=pool_size))
model4.add(Dropout(0.5))
model4.add(Flatten())
model4.add(Dense(128))
model4.add(Activation('relu'))
model4.add(Dense(43))
model4.add(Activation('softmax'))

model4.summary()

# TODO: Compile and train the model here.

# Parameters
nb_classes = 43                    # number of classes
nb_epoch = 20                      # number of epochs in training (sneaked from the solution ... 2 above is a typo?)
batch_size = 512                   # batch size (not given, so picked a decent one...)

# One-hot encoding of labels
Y_train = np_utils.to_categorical(y_train, nb_classes)   # note the upper case Y_ notation
Y_val = np_utils.to_categorical(y_val, nb_classes)

# Compile the model
model4.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model4.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_6 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
activation_18 (Activation)       (None, 30, 30, 32)    0           convolution2d_6[0][0]            
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 15, 15, 32)    0           activation_18[0][0]              
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_2[0][0]             
___________________________________________________________________________________________

In [21]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9735

Training 0.9737

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [22]:
# Part 1: recreate the data - copy of code above to reset from the loaded 'data' variable
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], train_size=0.8)
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255 - 0.5
X_val = X_val / 255 - 0.5
# Now have the data back as far as the normalised state

# Create the model
from keras.models import Sequential
from keras.layers import Convolution2D, Dense, Activation, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam  # import the Adam optimiser
from keras.utils import np_utils   # import some utils for the one-hot encoding

img_rows = 32
img_cols = 32
img_depth = 3
input_shape = (img_rows, img_cols, img_depth)       # parameter for the convolutional layer
kernel_size = (3, 3)                                # convolution kernel size
nb_filters = 32                                     # number of convolutional filters to use
pool_size = (2, 2)                                  # size of pooling area for max pooling

model5 = Sequential()
model5.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size=pool_size))
model5.add(Dropout(0.5))

input_shape_2 = (15,15,3)                           # just see if this works...
model5.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape_2))
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size=pool_size))
model5.add(Dropout(0.5))

model5.add(Flatten())
model5.add(Dense(128))
model5.add(Activation('relu'))
model5.add(Dense(72))                               # just another layer, reducing dimension again
model5.add(Activation('relu'))
model5.add(Dense(43))
model5.add(Activation('softmax'))

model5.summary()

# TODO: Compile and train the model here.

# Parameters
nb_classes = 43                    # number of classes
nb_epoch = 50                      # number of epochs in training 
batch_size = 512                   # batch size (not given, so picked a decent one...)

# One-hot encoding of labels
Y_train = np_utils.to_categorical(y_train, nb_classes)   # note the upper case Y_ notation
Y_val = np_utils.to_categorical(y_val, nb_classes)

# Compile the model
model5.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model5.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_7[0][0]      
____________________________________________________________________________________________________
activation_21 (Activation)       (None, 30, 30, 32)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)    (None, 15, 15, 32)    0           activation_21[0][0]              
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_3[0][0]             
___________________________________________________________________________________________

**Best Validation Accuracy:** 0.9931

Training accuracy: 0.9675

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [24]:
# TODO: Load test data
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
# TODO: Preprocess data & one-hot encode the labels
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

# TODO: Evaluate model on test data
model5.evaluate(X_test, Y_test)
#model5.metrics_names()

12608/12630 [============================>.] - ETA: 0s   64/12630 [..............................] - ETA: 6s

[0.27791217314234257, 0.93198733174036352]

In [25]:
model5.metrics_names

['loss', 'acc']

**Test Accuracy:** 0.932 accuracy

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.